# Import VILLESFR

In [1]:
import pandas as pd 
filepath = 'VILLESFR.json'
VILLESFR = pd.read_json(filepath, orient='records', lines=True)

# NER

In [12]:
import spacy
from spacy.tokens import DocBin, Doc
modelpath = '../resources/fr_spacy_custom_spancat_edda'
spacy_model = spacy.load(modelpath)
Doc.set_extension("metadata", default={}, force=True)

In [15]:
from transformers import PreTrainedModel, PretrainedConfig, Pipeline

class CustomModel(PreTrainedModel):
    def __init__(self, config: PretrainedConfig):
        super().__init__(config)

    def forward(self, inputs, **kwargs):
        spacydoc = spacy_model(inputs)
        spacydict = dict(spacydoc._.metadata, **spacydoc.to_json())
        return spacydict
    
    def ner_predict(self, inputs):
        spacydoc = spacy_model(inputs)
        spacydict = dict(spacydoc._.metadata, **spacydoc.to_json())
        return spacydict
    
class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs, maybe_arg=2):
        return inputs

    def _forward(self, model_inputs):
        model_outputs = self.model.forward(model_inputs)
        return model_outputs

    def postprocess(self, model_outputs):
        """
        transforms my dict into a list of dict :
        [{'entity': 'O',
        'score': 0.9999761581420898,
        'word': 'A',
        'start': 0,
        'end': 1},
        ... ,
        {'entity': 'B-LOC',
        'score': 0.9996446371078491,
        'word': 'Sheffield',
        'start': 74,
        'end': 83},
        {'entity': 'O',
        'score': 0.9999758005142212,
        'word': '.',
        'start': 83,
        'end': 84}]
        """
        output_list = []
        for span in model_outputs['spans']['sc']:
            token = model_outputs['text'][span['start']:span['end']]
            
            # remove leading lowercase characters, symbols and spaces
            shift = 0
            if span['label'] in ['NP-Spatial', 'Head']:
                while token[0].islower() or not token[0].isalpha() or token[0] == ' ':
                    token = token[1:]
                    shift += 1
                span_dict = {'entity': 'B-LOC',
                            'score': 1,
                            'word': token,
                            'start': span['start'] + shift,
                            'end': span['end']
                            }
                output_list.append(span_dict)
        return output_list

In [19]:
my_ner_model = CustomModel(config = PretrainedConfig())  
my_ner_pipeline = MyPipeline(model=my_ner_model,
                             villesfr = True
                             ) 
sentence = "* ALBI, (Géog.) ville de France, capitale de l'Albigeois, dans le haut Languedoc : elle est sur le Tarn. Long. 19. 49. lat. 43. 55. 44."
rmentions = my_ner_pipeline(sentence)
rmentions

[{'entity': 'B-LOC', 'score': 1, 'word': 'France', 'start': 25, 'end': 31},
 {'entity': 'B-LOC', 'score': 1, 'word': 'Albigeois', 'start': 47, 'end': 56},
 {'entity': 'B-LOC', 'score': 1, 'word': 'Tarn', 'start': 99, 'end': 103},
 {'entity': 'B-LOC', 'score': 1, 'word': 'Languedoc', 'start': 71, 'end': 80}]

# Ranker

In [12]:
from pathlib import Path
path = '/home/antoine/Documents/GitHub/T-Res/'
from t_res.geoparser import ranking

In [20]:
# --------------------------------------
# Instantiate the ranker:
myranker = ranking.Ranker(
    method="deezymatch",
    resources_path= path + "resources/",
    strvar_parameters={"overwrite_dataset": False,},
    deezy_parameters={
        # Paths and filenames of DeezyMatch models and data:
        "dm_path": str(Path( path + "resources/deezymatch/").resolve()),
        "dm_cands": "wkdtalts",
        "dm_model": "w2v_ocr",
        "dm_output": "deezymatch_on_the_fly",
        # Ranking measures:
        "ranking_metric": "faiss",
        "selection_threshold": 50,
        "num_candidates": 10,
        "verbose": False,
        # DeezyMatch training:
        "overwrite_training": False,
        "do_test": False,
    },
)

In [14]:
# Load the resources:
myranker.mentions_to_wikidata = myranker.load_resources()

*** Loading the ranker resources.


In [21]:
myranker.deezy_on_the_fly(['ROQUEMADOUR'])[0]

{'ROQUEMADOUR': {'Roquedur': {'Score': 0.40767000000000003,
   'Candidates': {'Q863558': 1.0}},
  'Corredor': {'Score': 0.249824,
   'Candidates': {'Q3694385': 1.0, 'Q6745839': 1.0}},
  'Oradour': {'Score': 0.22409999999999997,
   'Candidates': {'Q836897': 0.04477611940298507,
    'Q190974': 0.05172413793103448,
    'Q807034': 1.0}},
  'Broomhead Reservoir': {'Score': 0.209354,
   'Candidates': {'Q62595333': 0.6666666666666666}},
  'Roman emperor': {'Score': 0.17211600000000005,
   'Candidates': {'Q2277': 0.0004914349901713002,
    'Q12544': 7.203054094936253e-05}},
  'Békourou': {'Score': 0.16608000000000003, 'Candidates': {'Q55606267': 1.0}},
  'Rosemoor': {'Score': 0.16562799999999997,
   'Candidates': {'Q3442463': 0.009708737864077669,
    'Q7368490': 0.2631578947368421}},
  'Oredo': {'Score': 0.16227000000000003,
   'Candidates': {'Q7101001': 0.8823529411764706}},
  "Centre de la mémoire d'Oradour": {'Score': 0.14904200000000004,
   'Candidates': {'Q442113': 1.0}},
  'Pendour': {'

# T-RES using DeezyMatch with REL disambiguation

REL disambiguation **filtering out microtoponyms** and without adding place of publication.

In [2]:
import os
import sys
import sqlite3
from pathlib import Path
import json
from t_res.geoparser import geode_pipe,ranking,linking
path = '/home/antoine/Documents/GitHub/T-Res/'
NER_path = '../resources/fr_spacy_custom_spancat_edda'

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
# --------------------------------------
# Instantiate the ranker:
myranker = ranking.Ranker(
    method="deezymatch",
    resources_path= path + "resources/",
    strvar_parameters={"overwrite_dataset": False,},
    deezy_parameters={
        # Paths and filenames of DeezyMatch models and data:
        "dm_path": str(Path( path + "resources/deezymatch/").resolve()),
        "dm_cands": "wkdtalts",
        "dm_model": "w2v_ocr",
        "dm_output": "deezymatch_on_the_fly",
        # Ranking measures:
        "ranking_metric": "faiss",
        "selection_threshold": 50,
        "num_candidates": 10,
        "verbose": False,
        # DeezyMatch training:
        "overwrite_training": False,
        "do_test": False,
    },
)

# --------------------------------------
# Instantiate the Linker:
with sqlite3.connect( path + "resources/rel_db/embeddings_database.db") as conn:
    cursor = conn.cursor()
    mylinker = linking.Linker(
        method="reldisamb",
        resources_path= path + "resources/",
        rel_params={
            "model_path": path + "resources/models/disambiguation/",
            "data_path":  path + "experiments/outputs/data/lwm/",
            "training_split": "originalsplit",
            "db_embeddings": cursor,
            "with_publication": False,
            "without_microtoponyms": True,
            "do_test": False,
            "default_publname": "",
            "default_publwqid": "",
        },
        overwrite_training=False,
    )

In [4]:
geoparser = geode_pipe.Pipeline(geodeNERpath=NER_path,
                              myranker=myranker,
                              mylinker=mylinker)

*** Loading the ranker resources.
The DeezyMatch model is already trained!
*** Load linking resources.
  > Loading mentions to wikidata mapping.
  > Loading gazetteer.
*** Linking resources loaded!

Model path: /home/antoine/Documents/GitHub/T-Res/resources/models/disambiguation/deezymatch+10+50_originalsplit+wmtops/model eval
Loading model from given path: /home/antoine/Documents/GitHub/T-Res/resources/models/disambiguation/deezymatch+10+50_originalsplit+wmtops/model
{'mode': 'eval', 'model_path': '/home/antoine/Documents/GitHub/T-Res/resources/models/disambiguation/deezymatch+10+50_originalsplit+wmtops/model', 'prerank_ctx_window': 50, 'keep_p_e_m': 4, 'keep_ctx_ent': 3, 'ctx_window': 100, 'tok_top_n': 25, 'mulrel_type': 'ment-norm', 'n_rels': 3, 'hid_dims': 100, 'emb_dims': 300, 'snd_local_ctx_window': 6, 'dropout_rate': 0.3, 'n_epochs': 1000, 'dev_f1_change_lr': 0.915, 'n_not_inc': 10, 'eval_after_n_epochs': 5, 'learning_rate': 0.0001, 'margin': 0.01, 'df': 0.5, 'n_loops': 10, 'n_c

In [5]:
resolved = geoparser.run_text("A remarkable case of rattening has just occurred in the building trade next to the Market-street of Shefrield, but also in Lancaster. Not in Nottingham though. Not in Ashton either, nor in Salop!")
    
for r in resolved:
    print(json.dumps(r, indent=2))
    print()

  0%|          | 0/1 [00:00<?, ?it/s]

{
  "mention": "Shefrield",
  "ner_score": 1,
  "pos": 100,
  "sent_idx": 0,
  "end_pos": 109,
  "tag": "B-LOC",
  "sentence": "A remarkable case of rattening has just occurred in the building trade next to the Market-street of Shefrield, but also in Lancaster.",
  "prediction": "Q42448",
  "ed_score": 0.005,
  "cross_cand_score": {
    "Q42448": 0.664,
    "Q1862179": 0.144,
    "Q5953687": 0.071,
    "Q3461338": 0.066,
    "Q7492568": 0.028,
    "Q7492567": 0.028,
    "Q665346": 0.0
  },
  "string_match_score": {
    "Sherfield": [
      0.859,
      [
        "Q2258424"
      ]
    ],
    "Shenfield": [
      0.708,
      [
        "Q3461338",
        "Q665346"
      ]
    ],
    "Sheffield": [
      0.659,
      [
        "Q6707254",
        "Q823917",
        "Q5953687",
        "Q7492778",
        "Q1421317",
        "Q7492594",
        "Q897533",
        "Q42448",
        "Q7492565",
        "Q1862179",
        "Q4834926",
        "Q17643392",
        "Q7492570",
        "Q19509

In [6]:
sentence = "* ALBI, (Géog.) ville de France, capitale de l'Albigeois, dans le haut Languedoc : elle est sur le Tarn. Long. 19. 49. lat. 43. 55. 44."
sentence.find('ALBI')
resolved = geoparser.run_sentence(sentence, HEAD='ALBI', verbose=True)
for r in resolved:
    print(r)

  0%|          | 0/1 [00:00<?, ?it/s]

mentions : [{'entity': 'B-LOC', 'score': 1, 'mention': 'ALBI', 'start': 2, 'end': 5}, {'entity': 'B-LOC', 'score': 1, 'mention': 'France', 'start': 25, 'end': 31}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Albigeois', 'start': 47, 'end': 56}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Tarn', 'start': 99, 'end': 103}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Languedoc', 'start': 71, 'end': 80}]
rmentions : [{'mention': 'ALBI'}, {'mention': 'France'}, {'mention': 'Albigeois'}, {'mention': 'Tarn'}, {'mention': 'Languedoc'}]
wk_cands : {'Albigeois': {'Albigeois': {'Score': 1.0, 'Candidates': {'Q132801': 0.050890585241730284}}}, 'Tarn': {'Tarn': {'Score': 1.0, 'Candidates': {'Q215653': 0.8135593220338984, 'Q12772': 0.9555921052631579, 'Q1538251': 0.08695652173913043}}}, 'Languedoc': {'Languedoc': {'Score': 1.0, 'Candidates': {'Q206221': 0.9739336492890996, 'Q17005': 0.009146341463414635, 'Q1886509': 1.0}}}, 'ALBI': {'Albi': {'Score': 1.0, 'Candidates': {'Q94744': 0.07142857142857142, 

# Performance

In [21]:
import pandas as pd
import WikidataObject as wdo

sample2 = VILLESFR.copy(deep=True)
verbose = False

if 'resolved' not in sample2.columns:
    sample2['resolved'] = None

for i, row in sample2.iterrows():
    resolved = geoparser.run_sentence(row['fullcontent'], HEAD=row['head'])

    skyline = row['gold'] in resolved[0]['cross_cand_score'].keys()
    best_pred = resolved[0]['prediction']

    if best_pred[0] != 'Q':
        acc10 = False
    else:
        wd_pred = wdo.WikidataObject(best_pred, coordinates=resolved[0]['latlon'])
        acc10 = wd_pred._distance_to(row['gold']) <= 10

    sample2.at[i, 'resolved'] = resolved[0]
    sample2.at[i, 'skyline'] = skyline
    sample2.at[i, 'bestPred'] = best_pred
    sample2.at[i, 'acc10'] = acc10

    if verbose:
        print(f"Head: {row['head']}")
        print(f"Gold: {row['gold']}")
        print(f"Prediction: {best_pred}")
        print(f"Skyline: {skyline}")
        print(resolved[0]['cross_cand_score'])
        print(f"Accuracy 10: {acc10}\n")


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
sample2.to_json('t_res_results_140424.json', orient='records', lines=True)
print(sample2.skyline.value_counts(normalize=True), '\n')
print(sample2.acc10.value_counts(normalize=True), '\n')

# Perfect match

In [7]:
import os
import sys
import sqlite3
from pathlib import Path
import json
from t_res.geoparser import geode_pipe,ranking,linking
path = '/home/antoine/Documents/GitHub/T-Res/'
NER_path = path + 'resources/fr_spacy_custom_spancat_edda'

In [18]:
myranker = ranking.Ranker(
    method="perfectmatch",
    resources_path="../resources/",
)

In [11]:
mylinker = linking.Linker(
    method="mostpopular",
    resources_path="../resources/",
)

In [12]:
geoparser = geode_pipe.Pipeline(geodeNERpath=NER_path,
                              myranker=myranker,
                              mylinker=mylinker)

*** Loading the ranker resources.
*** Load linking resources.
  > Loading mentions to wikidata mapping.
  > Loading gazetteer.
*** Linking resources loaded!



In [13]:
sentence = "* ALBI, (Géog.) ville de France, capitale de l'Albigeois, dans le haut Languedoc : elle est sur le Tarn. Long. 19. 49. lat. 43. 55. 44."
sentence.find('ALBI')
resolved = geoparser.run_sentence(sentence, HEAD='ALBI', verbose=True)
for r in resolved:
    print(r)

mentions : [{'entity': 'B-LOC', 'score': 1, 'mention': 'ALBI', 'start': 2, 'end': 5}, {'entity': 'B-LOC', 'score': 1, 'mention': 'France', 'start': 25, 'end': 31}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Albigeois', 'start': 47, 'end': 56}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Tarn', 'start': 99, 'end': 103}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Languedoc', 'start': 71, 'end': 80}]
rmentions : [{'mention': 'ALBI'}, {'mention': 'France'}, {'mention': 'Albigeois'}, {'mention': 'Tarn'}, {'mention': 'Languedoc'}]
wk_cands : {'France': {'France': {'Score': 1.0, 'Candidates': {'Q71084': 0.10598896769109535, 'Q146246': 0.00429553264604811, 'Q71092': 0.12113564668769716, 'Q58296': 0.09373020899303357, 'Q70802': 0.3776297335203366, 'Q58326': 0.031683168316831684, 'Q69808': 0.011072103699702943, 'Q185682': 0.0009886307464162135, 'Q142': 0.8872503121098627, 'Q170604': 0.002657218777679362, 'Q2841718': 0.044444444444444446, 'Q271894': 0.0023923444976076554, 'Q218272': 0.000687285

In [14]:
import pandas as pd
import WikidataObject as wdo

In [19]:
sample3 = VILLESFR.copy(deep=True)
verbose = True

if 'resolved' not in sample3.columns:
    sample['resolved'] = None

for i, row in sample.iterrows():
    resolved = geoparser.run_sentence(row['fullcontent'], HEAD=row['head'])

    skyline = row['gold'] in resolved[0]['cross_cand_score'].keys()
    best_pred = resolved[0]['prediction']

    if best_pred[0] != 'Q':
        acc10 = False
    else:
        wd_pred = wdo.WikidataObject(best_pred, coordinates=resolved[0]['latlon'])
        acc10 = wd_pred._distance_to(row['gold']) <= 10

    sample.at[i, 'resolved'] = resolved[0]
    sample.at[i, 'skyline'] = skyline
    sample.at[i, 'bestPred'] = best_pred
    sample.at[i, 'acc10'] = acc10

    if verbose:
        print(f"Head: {row['head']}")
        print(f"Gold: {row['gold']}")
        print(f"Prediction: {best_pred}")
        print(f"Skyline: {skyline}")
        print(resolved[0]['cross_cand_score'])
        print(f"Accuracy 10: {acc10}\n")


Head: Afrique
Gold: Q818608
Prediction: NIL
Skyline: False
{}
Accuracy 10: False

Head: AGDE
Gold: Q191396
Prediction: NIL
Skyline: False
{}
Accuracy 10: False

Head: AGEN
Gold: Q6625
Prediction: NIL
Skyline: False
{}
Accuracy 10: False

Head: AGRERE
Gold: Q137514
Prediction: NIL
Skyline: False
{}
Accuracy 10: False

Head: Aigle
Gold: Q500588
Prediction: Q43195
Skyline: False
{'Q43195': 0.782, 'Q647923': 0.208, 'Q1011820': 0.01}
Accuracy 10: False

Head: AIGNAI-LE-DUC
Gold: Q198315
Prediction: NIL
Skyline: False
{}
Accuracy 10: False

Head: AIGNAN (Saint)
Gold: Q832394
Prediction: NIL
Skyline: False
{}
Accuracy 10: False

Head: AIGREMONT-LE-DUC
Gold: Q1102347
Prediction: NIL
Skyline: False
{}
Accuracy 10: False

Head: AIGUES-MORTES
Gold: Q193809
Prediction: NIL
Skyline: False
{}
Accuracy 10: False

Head: AIGLE-PERSE
Gold: Q333447
Prediction: NIL
Skyline: False
{}
Accuracy 10: False

Head: Aiguillon
Gold: Q404783
Prediction: Q404783
Skyline: True
{'Q404783': 0.8, 'Q404757': 0.133, 'Q605

In [20]:
sample.to_json('t_res_perfect+mostpop_140424.json', orient='records', lines=True)
print(sample.skyline.value_counts(normalize=True), '\n')
print(sample.acc10.value_counts(normalize=True), '\n')

False    0.938257
True     0.061743
Name: skyline, dtype: float64 

False    0.945521
True     0.054479
Name: acc10, dtype: float64 



# Partial match

In [2]:
import os
import sys
import sqlite3
from pathlib import Path
import json
from t_res.geoparser import geode_pipe,ranking,linking
path = '/home/antoine/Documents/GitHub/T-Res/'
NER_path = path + 'resources/fr_spacy_custom_spancat_edda'

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
myranker = ranking.Ranker(
    method="partialmatch",
    resources_path="../resources/",
)

In [4]:
mylinker = linking.Linker(
    method="mostpopular",
    resources_path="../resources/",
)

In [5]:
geoparser = geode_pipe.Pipeline(geodeNERpath=NER_path,
                              myranker=myranker,
                              mylinker=mylinker)

*** Loading the ranker resources.
INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
*** Load linking resources.
  > Loading mentions to wikidata mapping.
  > Loading gazetteer.
*** Linking resources loaded!



In [6]:
sentence = "* ALBI, (Géog.) ville de France, capitale de l'Albigeois, dans le haut Languedoc : elle est sur le Tarn. Long. 19. 49. lat. 43. 55. 44."
sentence.find('ALBI')
resolved = geoparser.run_sentence(sentence, HEAD='ALBI', verbose=True)
for r in resolved:
    print(r)

mentions : [{'entity': 'B-LOC', 'score': 1, 'mention': 'ALBI', 'start': 2, 'end': 5}, {'entity': 'B-LOC', 'score': 1, 'mention': 'France', 'start': 25, 'end': 31}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Albigeois', 'start': 47, 'end': 56}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Tarn', 'start': 99, 'end': 103}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Languedoc', 'start': 71, 'end': 80}]
rmentions : [{'mention': 'ALBI'}, {'mention': 'France'}, {'mention': 'Albigeois'}, {'mention': 'Tarn'}, {'mention': 'Languedoc'}]
wk_cands : {'Languedoc': {'Languedoc': {'Score': 1.0, 'Candidates': {'Q206221': 0.9739336492890996, 'Q17005': 0.009146341463414635, 'Q1886509': 1.0}}}, 'Albigeois': {'Albigeois': {'Score': 1.0, 'Candidates': {'Q132801': 0.050890585241730284}}}, 'ALBI': {'Albi': {'Score': 1.0, 'Candidates': {'Q94744': 0.07142857142857142, 'Q132801': 0.9211195928753181, 'Q951428': 0.3793103448275862, 'Q53749': 1.0, 'Q284698': 0.16666666666666666, 'Q700379': 0.4}}}, 'France': {'Fran

In [7]:
import pandas as pd
import WikidataObject as wdo

In [9]:
sample3 = VILLESFR.sample(5) #.copy(deep=True)
verbose = True

if 'resolved' not in sample3.columns:
    sample3['resolved'] = None

for i, row in sample3.iterrows():
    resolved = geoparser.run_sentence(row['fullcontent'], HEAD=row['head'])

    skyline = row['gold'] in resolved[0]['cross_cand_score'].keys()
    best_pred = resolved[0]['prediction']

    if best_pred[0] != 'Q':
        acc10 = False
    else:
        wd_pred = wdo.WikidataObject(best_pred, coordinates=resolved[0]['latlon'])
        acc10 = wd_pred._distance_to(row['gold']) <= 10

    sample3.at[i, 'resolved'] = resolved[0]
    sample3.at[i, 'skyline'] = skyline
    sample3.at[i, 'bestPred'] = best_pred
    sample3.at[i, 'acc10'] = acc10

    if verbose:
        print(f"Head: {row['head']}")
        print(f"Gold: {row['gold']}")
        print(f"Prediction: {best_pred}")
        print(f"Skyline: {skyline}")
        print(resolved[0]['cross_cand_score'])
        print(f"Accuracy 10: {acc10}\n")


Head: RETHEL
Gold: Q209790
Prediction: Q209790
Skyline: True
{'Q209790': 0.974, 'Q702199': 0.026}
Accuracy 10: True

Head: BRIGNOLES
Gold: Q207584
Prediction: Q207584
Skyline: True
{'Q207584': 0.973, 'Q700755': 0.027}
Accuracy 10: True

Head: AUTUN
Gold: Q190827
Prediction: Q190827
Skyline: True
{'Q190827': 0.982, 'Q611944': 0.009, 'Q662088': 0.006, 'Q444201': 0.003}
Accuracy 10: True

Head: Brie-Comte-Braine
Gold: Q332052
Prediction: Q332052
Skyline: True
{'Q332052': 0.941, 'Q845920': 0.029, 'Q95129': 0.029}
Accuracy 10: True

Head: Chateau-Salins
Gold: Q22745
Prediction: Q22745
Skyline: True
{'Q22745': 0.875, 'Q700954': 0.125}
Accuracy 10: True



In [ ]:
sample3.to_json('t_res_perfect+mostpop_140424.json', orient='records', lines=True)
print(sample3.skyline.value_counts(normalize=True), '\n')
print(sample3.acc10.value_counts(normalize=True), '\n')